In [262]:
import warnings

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SVMSMOTE
from sklearn.feature_selection import SelectKBest, mutual_info_classif

### 2. Загрузим и выведем информацию о тренировочных и тестовых данных 

In [263]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Просмотр первых строк данных
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3169 entries, 0 to 3168
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        3169 non-null   int64  
 1   ClientPeriod              3169 non-null   int64  
 2   MonthlySpending           3169 non-null   float64
 3   TotalSpent                3169 non-null   object 
 4   Sex                       3169 non-null   object 
 5   IsSeniorCitizen           3169 non-null   int64  
 6   HasPartner                3169 non-null   object 
 7   HasChild                  3169 non-null   object 
 8   HasPhoneService           3169 non-null   object 
 9   HasMultiplePhoneNumbers   3169 non-null   object 
 10  HasInternetService        3169 non-null   object 
 11  HasOnlineSecurityService  3169 non-null   object 
 12  HasOnlineBackup           3169 non-null   object 
 13  HasDeviceProtection       3169 non-null   object 
 14  HasTechS

In [264]:
test_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2113 entries, 0 to 2112
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        2113 non-null   int64  
 1   ClientPeriod              2113 non-null   int64  
 2   MonthlySpending           2113 non-null   float64
 3   TotalSpent                2113 non-null   object 
 4   Sex                       2113 non-null   object 
 5   IsSeniorCitizen           2113 non-null   int64  
 6   HasPartner                2113 non-null   object 
 7   HasChild                  2113 non-null   object 
 8   HasPhoneService           2113 non-null   object 
 9   HasMultiplePhoneNumbers   2113 non-null   object 
 10  HasInternetService        2113 non-null   object 
 11  HasOnlineSecurityService  2113 non-null   object 
 12  HasOnlineBackup           2113 non-null   object 
 13  HasDeviceProtection       2113 non-null   object 
 14  HasTechS

### 3. Предварительная обработка данных 

In [265]:
ids = test_data["id"]

# Преобразование TotalSpent в числовой формат
test_data['TotalSpent'] = pd.to_numeric(test_data['TotalSpent'], errors='coerce')
train_data['TotalSpent'] = pd.to_numeric(train_data['TotalSpent'], errors='coerce')


# Преобразование бинарных переменных Yes/No в 1/0
binary_columns = ['HasPartner', 'HasChild', 'HasPhoneService', 'HasMultiplePhoneNumbers', 
                 'IsBillingPaperless']

for col in binary_columns:
    train_data[col] = train_data[col].map({'Yes': 1, 'No': 0})
    test_data[col] = test_data[col].map({'Yes': 1, 'No': 0})
    

# Выборка только нужных категориальных переменных
categorical_columns = ['Sex', 'HasPartner', 'HasChild', 'HasPhoneService', 
                       'HasMultiplePhoneNumbers', 'HasInternetService', 
                       'HasOnlineSecurityService', 'HasOnlineBackup', 
                       'HasDeviceProtection', 'HasTechSupportAccess', 
                       'HasOnlineTV', 'HasMovieSubscription', 
                       'HasContractPhone', 'IsBillingPaperless', 
                       'PaymentMethod']

train_data = pd.get_dummies(train_data, columns=categorical_columns, drop_first=True)
test_data = pd.get_dummies(test_data, columns=categorical_columns, drop_first=True)


# Проверка на наличие пропущенных значений
print(train_data.isnull().sum())

# Заполнение пропущенных значений (например, средним значением для числовых столбцов)
x = train_data.fillna(train_data.mean())

# Проверка на наличие пропущенных значений
print(test_data.isnull().sum())

# Заполнение пропущенных значений (например, средним значением для числовых столбцов)
x = test_data.fillna(test_data.mean())

train_data.head()


id                                              0
ClientPeriod                                    0
MonthlySpending                                 0
TotalSpent                                      6
IsSeniorCitizen                                 0
Churn                                           0
Sex_Male                                        0
HasPartner_1                                    0
HasChild_1                                      0
HasPhoneService_1                               0
HasMultiplePhoneNumbers_1.0                     0
HasInternetService_Fiber optic                  0
HasInternetService_No                           0
HasOnlineSecurityService_No internet service    0
HasOnlineSecurityService_Yes                    0
HasOnlineBackup_No internet service             0
HasOnlineBackup_Yes                             0
HasDeviceProtection_No internet service         0
HasDeviceProtection_Yes                         0
HasTechSupportAccess_No internet service        0


,id,ClientPeriod,MonthlySpending,TotalSpent,IsSeniorCitizen,Churn,Sex_Male,HasPartner_1,HasChild_1,HasPhoneService_1,...,HasOnlineTV_No internet service,HasOnlineTV_Yes,HasMovieSubscription_No internet service,HasMovieSubscription_Yes,HasContractPhone_One year,HasContractPhone_Two year,IsBillingPaperless_1,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,4719,28,20.30,487.95,0,0,True,True,True,True,...,True,False,True,False,False,True,True,True,False,False
1,3190,12,81.45,912.00,0,0,True,False,False,True,...,False,True,False,False,False,False,True,False,True,False
2,5215,25,20.15,536.35,0,0,True,False,False,True,...,True,False,True,False,False,False,True,False,False,True
3,1472,50,69.50,3418.20,0,0,True,False,False,True,...,False,False,False,True,False,True,False,False,False,False
4,4413,39,101.25,3949.15,1,0,False,True,False,True,...,False,True,False,True,False,False,True,False,True,False


In [266]:
train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3169 entries, 0 to 3168
Data columns (total 31 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            3169 non-null   int64  
 1   ClientPeriod                                  3169 non-null   int64  
 2   MonthlySpending                               3169 non-null   float64
 3   TotalSpent                                    3163 non-null   float64
 4   IsSeniorCitizen                               3169 non-null   int64  
 5   Churn                                         3169 non-null   int64  
 6   Sex_Male                                      3169 non-null   bool   
 7   HasPartner_1                                  3169 non-null   bool   
 8   HasChild_1                                    3169 non-null   bool   
 9   HasPhoneService_1                             3169 non-null   b

In [267]:
# Разделение данных на признаки и целевую переменную
key = 'Churn'
x = train_data.drop([key], axis=1)
y = train_data[key]

# Разделение данных на обучающую и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

# Масштабирование данных
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Теперь x_train_scaled и x_test_scaled готовы для использования в модели

In [268]:
# Проверка на наличие пропущенных значений
print(x.isnull().sum())

# Заполнение пропущенных значений (например, средним значением для числовых столбцов)
x = x.fillna(x.mean())

# Выбор признаков с помощью SelectKBest
feature_names = list(train_data.columns)[:-1]
sel_mi = SelectKBest(mutual_info_classif, k=8).fit(x, y)
features = list(zip(feature_names, sel_mi.get_support()))
print(features)

# Отбор полезных признаков
useful_features = [i[0] for i in features if i[1] == True]
print(useful_features)

# Создание нового набора данных с отобранными признаками
x_new = x[useful_features].copy(deep=True)

# Разделение данных на обучающую и тестовую выборки
x_train_new, x_test_new, y_train_new, y_test_new = train_test_split(x_new, y, test_size=0.3)

# Масштабирование данных
scaler = StandardScaler()
x_train_new = scaler.fit_transform(x_train_new)
x_test_new = scaler.transform(x_test_new)

# Просмотр первых строк новых данных
print(pd.DataFrame(x_train_new, columns=useful_features).head())



id                                              0
ClientPeriod                                    0
MonthlySpending                                 0
TotalSpent                                      6
IsSeniorCitizen                                 0
Sex_Male                                        0
HasPartner_1                                    0
HasChild_1                                      0
HasPhoneService_1                               0
HasMultiplePhoneNumbers_1.0                     0
HasInternetService_Fiber optic                  0
HasInternetService_No                           0
HasOnlineSecurityService_No internet service    0
HasOnlineSecurityService_Yes                    0
HasOnlineBackup_No internet service             0
HasOnlineBackup_Yes                             0
HasDeviceProtection_No internet service         0
HasDeviceProtection_Yes                         0
HasTechSupportAccess_No internet service        0
HasTechSupportAccess_Yes                        0


### 4. Построение модели

In [269]:
models = {
    'Логистическая Регрессия': LogisticRegression(solver='liblinear'),
    'Метод Ближайших Соседей': KNeighborsClassifier(),
    'Наивный Байесовский Классификатор': GaussianNB(),
    'Линейный Дискриминантный Анализ': LinearDiscriminantAnalysis(),
    'Квадратичный Дискриминантный Анализ': QuadraticDiscriminantAnalysis(),
    'Машина Опорных Векторов': SVC()
}

params = {
    'Логистическая Регрессия': {"max_iter": [100],
                                 "solver": ['sag', 'saga', 'newton-cg'],
                                 "n_jobs": [-1]},
    'Метод Ближайших Соседей': {"n_neighbors": list(range(3, 7)),
                                 "weights": ["uniform", "distance"],
                                 "p": [1, 2, 3, 4]},  
    'Наивный Байесовский Классификатор': {},
    'Линейный Дискриминантный Анализ': {"solver":  ["svd", 'lsqr', "eigen"]},
    'Квадратичный Дискриминантный Анализ': {"reg_param": np.linspace(0, 1, num=101)},
    'Машина Опорных Векторов': {"kernel": ['poly'],
                                'probability': [True],
                                 "class_weight": ["balanced"],
                                 "max_iter": [-1]}
}

# Поиск лучших моделей
best_models = {}
for name, model in models.items():
    random_search = GridSearchCV(model, params[name], cv=5, scoring='roc_auc', n_jobs=-1)
    random_search.fit(x_train_new, y_train_new)
    best_model = random_search.best_estimator_
    best_models[name] = best_model
    print(f"Лучшие параметры для {name}: {random_search.best_params_}")
   

Лучшие параметры для Логистическая Регрессия: {'max_iter': 100, 'n_jobs': -1, 'solver': 'saga'}
Лучшие параметры для Метод Ближайших Соседей: {'n_neighbors': 6, 'p': 2, 'weights': 'uniform'}
Лучшие параметры для Наивный Байесовский Классификатор: {}
Лучшие параметры для Линейный Дискриминантный Анализ: {'solver': 'svd'}
Лучшие параметры для Квадратичный Дискриминантный Анализ: {'reg_param': 0.02}
Лучшие параметры для Машина Опорных Векторов: {'class_weight': 'balanced', 'kernel': 'poly', 'max_iter': -1, 'probability': True}


In [270]:
print(best_models)

{'Логистическая Регрессия': LogisticRegression(n_jobs=-1, solver='saga'), 'Метод Ближайших Соседей': KNeighborsClassifier(n_neighbors=6), 'Наивный Байесовский Классификатор': GaussianNB(), 'Линейный Дискриминантный Анализ': LinearDiscriminantAnalysis(), 'Квадратичный Дискриминантный Анализ': QuadraticDiscriminantAnalysis(reg_param=0.02), 'Машина Опорных Векторов': SVC(class_weight='balanced', kernel='poly', probability=True)}


### 6. Сэмплирование 

In [271]:
# Инициализация SVMSMOTE
smote = SVMSMOTE(
    sampling_strategy='auto',  # Балансировка minority класса
    random_state=1,           # Для воспроизводимости
    k_neighbors=5,            # Количество соседей для создания синтетических примеров
    m_neighbors=10,           # Количество соседей для определения границы
    svm_estimator=SVC(probability=True)  # SVM для SVMSMOTE
)

# Применение SMOTE к данным
smoted_x, smoted_y = smote.fit_resample(x_new, y)

# Словарь для хранения результатов
results = {}

# Обучение и оценка моделей
for name, model in best_models.items():
    # Обучение модели на сбалансированных данных
    model.fit(smoted_x, smoted_y)
    
    # Предсказание вероятностей (если модель поддерживает predict_proba)
    if hasattr(model, "predict_proba"):
        y_pred = model.predict_proba(smoted_x)[:, 1]  # Оценка на сбалансированных данных
    else:
        y_pred = model.decision_function(smoted_x)  # Для моделей без predict_proba
    
    # Оценка ROC-AUC на сбалансированных данных
    score = roc_auc_score(smoted_y, y_pred)
    results[name] = score

# Вывод результатов
print(results)

C:\Users\sh764\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'Логистическая Регрессия': 0.7949919629308507, 'Метод Ближайших Соседей': 0.9256069190141072, 'Наивный Байесовский Классификатор': 0.7824283618666648, 'Линейный Дискриминантный Анализ': 0.8264951723199614, 'Квадратичный Дискриминантный Анализ': 0.8267997310591473, 'Машина Опорных Векторов': 0.7244752768175308}


### 7. Целевая переменная

In [272]:
best_model_name = max(results, key=results.get)
best_model = best_models[best_model_name]
print("Лучшая модель:", best_model_name)


Лучшая модель: Метод Ближайших Соседей


In [274]:
# Применение get_dummies к test_data
test_data = pd.get_dummies(test_data)

# Убедимся, что test_data имеет те же столбцы, что и train_data
missing_cols = set(train_data.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0  # Добавляем отсутствующие столбцы с нулевыми значениями

# Убедимся, что порядок столбцов совпадает
test_data = test_data[train_data.columns]

# Заполнение пропущенных значений
test_data[useful_features] = test_data[useful_features].fillna(test_data[useful_features].mean())

# Предсказание на тестовых данных
y_pred = best_model.predict_proba(test_data[useful_features])[:, 1]

# Создание файла submission
submission = pd.DataFrame({'id': test_data['id'], 'Churn': y_pred})
submission.to_csv('submission.csv', index=False)
print("Файл submission.csv сохранен.")


Файл submission.csv сохранен.
